In [15]:
import boto3
import sagemaker

In [16]:
import tarfile

# Comprimir el modelo en un archivo tar.gz
with tarfile.open("rf_model.tar.gz", "w:gz") as tar:
    tar.add("model/rf.pkl", arcname="rf.pkl")

In [17]:
import boto3

# Configura el bucket de S3
s3 = boto3.client('s3')
bucket_name = "travel-insurance-edvai"
model_path = "model/rf_model.tar.gz"

In [18]:
# Carga el modelo a S3
s3.upload_file("rf_model.tar.gz", bucket_name, model_path)
print(f"Modelo subido a s3://{bucket_name}/{model_path}")

Modelo subido a s3://travel-insurance-edvai/model/rf_model.tar.gz


In [22]:
from sagemaker.model import Model

# Establecer la URI de la imagen Docker
image_uri = "021406833539.dkr.ecr.us-east-1.amazonaws.com/rf-model:v2-json"

# Especificar la ubicación del modelo en S3
model_data = f"s3://{bucket_name}/{model_path}"

# Crear el modelo en SageMaker
role = "arn:aws:iam::021406833539:role/service-role/AmazonSageMaker-ExecutionRole-20251016T224544"
model = Model(image_uri=image_uri, model_data=model_data, role=role)

In [ ]:
# Desplegar el endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name="rf-model-endpoint"
)

In [25]:
import boto3

# Crear cliente de SageMaker Runtime
runtime = boto3.client('sagemaker-runtime')

payload = '{"Age": 35, "EmploymentType": 1, "GraduateOrNot": 1, "AnnualIncome": 50000, "FamilyMembers": 4, "ChronicDiseases": 0, "FrequentFlyer": 1, "EverTravelledAbroad": 1}'

# Enviar la solicitud al endpoint
response = runtime.invoke_endpoint(
    EndpointName="rf-model-endpoint",
    ContentType="application/json",
    Body=payload
)

# Imprimir la predicción
print("Predicción:", response['Body'].read().decode("utf-8"))

Predicción: {"score":0}
